In [ ]:
import ray
ray.init()

In [4]:
import ray
import random

@ray.remote
class StorageNode:
    def __init__(self, node_id):
        self.node_id = node_id
        self.chunks = {}

    def store_chunk(self, chunk_id, chunk_content):
        self.chunks[chunk_id] = chunk_content
        return True

    def retrieve_chunk(self, chunk_id):
        return self.chunks.get(chunk_id, None)


@ray.remote
class NameNode:
    def __init__(self):
        self.artifacts_metadata = {}
        self.storage_nodes = {}
        self.node_counter = 0

    def register_storage_node(self):
        node_id = self.node_counter
        new_node = StorageNode.remote(node_id)
        self.storage_nodes[node_id] = new_node
        self.node_counter += 1
        return node_id

    def upload_artifact(self, artifact_name, content, replication_factor=2):
        chunk_size = 1024  # Example chunk size
        chunks = [content[i:i+chunk_size] for i in range(0, len(content), chunk_size)]
        self.artifacts_metadata[artifact_name] = []
        for chunk_id, chunk in enumerate(chunks):
            stored_nodes = []
            for _ in range(replication_factor):
                node_id, storage_node = random.choice(list(self.storage_nodes.items()))
                ray.get(storage_node.store_chunk.remote(f"{artifact_name}_{chunk_id}", chunk))
                stored_nodes.append(node_id)
            self.artifacts_metadata[artifact_name].append((chunk_id, stored_nodes))
        return True

    def list_status(self):
        status = {}
        for artifact_name, chunks_info in self.artifacts_metadata.items():
            status[artifact_name] = {}
            for chunk_id, nodes in chunks_info:
                status[artifact_name][chunk_id] = nodes
        return status


# Initialize NameNode
name_node = NameNode.remote()

# Register some StorageNodes
for _ in range(5):  # Create 5 storage nodes
    name_node.register_storage_node.remote()

# Upload an artifact
artifact_name = "example_artifact"
artifact_content = "This is a simple example of storing an artifact in the distributed storage system."
ray.get(name_node.upload_artifact.remote(artifact_name, artifact_content))

# List the status of the cluster
status = ray.get(name_node.list_status.remote())
print("Cluster Status:", status)


(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: fffffffffffffffff6599556838bdba5d8eeca9801000000 Worker ID: 0d80cb952ce64209023c84f25261b15e7236488305f3ba23e9f18ab2 Node ID: 347b33641aa934828d1255e9b27df7adaa061ab8cd7779bc9f1b5276 Worker IP address: 127.0.0.1 Worker port: 64688 Worker PID: 6768 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly by a signal. SystemExit is raised (sys.exit is called). Exit code: 1. The process receives a SIGTERM.
Cluster Status: {'example_artifact': {0: [2, 2]}}
